In [1]:
import argparse
import torch
import torch.nn as nn
import numpy as np
import os
import pickle
import nltk
import pandas as pd
from torch.nn.utils.rnn import pack_padded_sequence
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
import csv
import torchvision.models as models
import torch.nn.functional as F
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import sentence_bleu
import timm
import pymeteor.pymeteor
from torchinfo import summary
from glob import glob
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from rouge_score import rouge_scorer
from torchvision.transforms import ToTensor
# Device configurationresul
tf = ToTensor()
device = torch.device('cuda:6' if torch.cuda.is_available() else 'cpu')

In [2]:
params={'image_size':512,
        'lr':2e-4,
        'beta1':0.5,
        'beta2':0.999,
        'batch_size':1,
        'epochs':10000,
        'data_path':'../../data/origin_type/',
        'train_csv':'BR_train.csv',
        'val_csv':'BR_test.csv',
        'vocab_path':'../../data/origin_type/BR_vocab.pkl',
        'embed_size':300,
        'hidden_size':256,
        'num_layers':1,}

In [3]:
class CustomDataset(Dataset):
    """COCO Custom Dataset compatible with torch.utils.data.DataLoader."""
    def __init__(self,data_list, data_path,image_size, csv, class_dataset, vocab, transform=None):
        """Set the path for images, captions and vocabulary wrapper.
        
        Args:
            root: image directory.
            json: coco annotation file path.
            vocab: vocabulary wrapper.
            transform: image transformer.
        """
        self.root = data_path+'**/**/'
        self.df = pd.read_csv(data_path+csv)
        self.class_dataset=class_dataset
        self.vocab = vocab
        self.transform = transform
        self.image_size=image_size
        self.data_list=data_list
    def __getitem__(self, index):
        """Returns one data pair (image and caption)."""
        df = self.df
        vocab = self.vocab
        img_id=df.loc[index]
        
        caption=img_id['caption']
        images = self.data_list[index]
        # Convert caption (string) to word ids.
        tokens = nltk.tokenize.word_tokenize(str(caption).lower())
        caption = []
        caption.append(vocab('<start>'))
        caption.extend([vocab(token) for token in tokens])
        caption.append(vocab('<end>'))
        target = torch.Tensor(caption)
        return images, target

    def __len__(self):
        return len(self.df)
    

class Vocabulary(object):
    """Simple vocabulary wrapper."""
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0

    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]

    def __len__(self):
        return len(self.word2idx)

def collate_fn(data):
    """Creates mini-batch tensors from the list of tuples (image, caption).
    
    We should build custom collate_fn rather than using default collate_fn, 
    because merging caption (including padding) is not supported in default.

    Args:
        data: list of tuple (image, caption). 
            - image: torch tensor of shape (3, 256, 256).
            - caption: torch tensor of shape (?); variable length.

    Returns:
        images: torch tensor of shape (batch_size, 3, 256, 256).
        targets: torch tensor of shape (batch_size, padded_length).
        lengths: list; valid length for each padded caption.
    """
    # Sort a data list by caption length (descending order).
    data.sort(key=lambda x: len(x[1]), reverse=True)
    images, captions = zip(*data)

    # Merge images (from tuple of 3D tensor to 4D tensor).
    images = torch.stack(images, 0)

    # Merge captions (from tuple of 1D tensor to 2D tensor).
    lengths = [len(cap) for cap in captions]
    targets = torch.zeros(len(captions), max(lengths)).long()
    for i, cap in enumerate(captions):
        end = lengths[i]
        targets[i, :end] = cap[:end]        
    return images, targets, lengths

def idx2word(vocab, indices):
    sentence = []
    
    aa=indices.cpu().numpy()
    
    for index in aa[0]:
        word = vocab.idx2word[index]
        sentence.append(word)
    return sentence
def word2sentence(words_list):
    sentence=''
    for word in words_list:
        if word.isalnum():
            sentence+=' '+word
        else:
            sentence+=word
    return sentence

In [4]:

class FeatureExtractor(nn.Module):
    """Feature extoractor block"""
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        cnn1= timm.create_model('maxvit_tiny_tf_512', pretrained=True)
        self.feature_ex = nn.Sequential(*list(cnn1.children())[:-1])

    def forward(self, inputs):
        features = self.feature_ex(inputs)
        
        return features
    
class AttentionMILModel(nn.Module):
    def __init__(self, num_classes, image_feature_dim,feature_extractor_scale1: FeatureExtractor):
        super(AttentionMILModel, self).__init__()
        self.num_classes = num_classes
        self.image_feature_dim = image_feature_dim

        # Remove the classification head of the CNN model
        self.feature_extractor = feature_extractor_scale1
        
        # Attention mechanism
        self.attention = nn.Sequential(
            nn.Linear(image_feature_dim, 128),
            nn.Tanh(),
            nn.Linear(128, 1)
        )
        
        # Classification layer
        self.classification_layer = nn.Linear(image_feature_dim, num_classes)

    def forward(self, inputs):
        batch_size, channels, height, width = inputs.size()
        
        # Flatten the inputs
        inputs = inputs.view(-1, channels, height, width)
        
        # Feature extraction using the pre-trained CNN
        features = self.feature_extractor(inputs)  # Shape: (batch_size , 2048, 1, 1)
        
        # Reshape features
        features = features.view(batch_size, -1)  # Shape: (batch_size, num_tiles, 2048)
        
        
        
        
        
        # Classification layer
        logits = self.classification_layer(features)  # Shape: (batch_size, num_classes)
        
        return logits  

class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=100):
        """Set the hyper-parameters and build the layers."""
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, num_layers, batch_first=True) # change for LSTM or RNN
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_seg_length = max_seq_length
        
    def forward(self, features, captions, lengths, teacher_forcing_ratio=0.5):
        """Decode image feature vectors and generates captions."""
        batch_size = features.size(0)
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        
        # Initialize hidden and cell states
        h, c = None, None
        
        outputs = torch.zeros(batch_size, captions.size(1), self.linear.out_features).to(features.device)
        
        # Iterate over the sequence length
        for t in range(captions.size(1)):
            if t == 0:
                # First input is the image features
                input_t = embeddings[:, t, :].unsqueeze(1)
            else:
                # Decide if we are going to use teacher forcing or not
                use_teacher_forcing = True if torch.rand(1).item() < teacher_forcing_ratio else False
                if use_teacher_forcing:
                    # Use the actual next word as the next input
                    input_t = embeddings[:, t, :].unsqueeze(1)
                else:
                    # Use the predicted word as the next input
                    input_t = self.embed(predicted).unsqueeze(1)
            
            # Get the LSTM outputs
            output, (h, c) = self.lstm(input_t, (h, c)) if h is not None else self.lstm(input_t)
            output = self.linear(output.squeeze(1))
            outputs[:, t, :] = output
            
            # Get the predicted word
            _, predicted = output.max(1)
        
        return outputs

    
    def sample(self, features, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []
        inputs = features.unsqueeze(1)
        for i in range(self.max_seg_length):
            hiddens, states = self.lstm(inputs, states)          # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))            # outputs:  (batch_size, vocab_size)
            _, predicted = outputs.max(1)                        # predicted: (batch_size)
            sampled_ids.append(predicted)

            inputs = self.embed(predicted)                       # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                         # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)                # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids
    
    def stochastic_sample(self, features, temperature, states=None):
        """Generate captions for given image features using greedy search."""
        sampled_ids = []
        inputs = features.unsqueeze(1)
        for i in range(self.max_seg_length):
            hiddens, states = self.lstm(inputs, states)          # hiddens: (batch_size, 1, hidden_size)
            outputs = self.linear(hiddens.squeeze(1))            # outputs:  (batch_size, vocab_size)
            
            soft_out = F.softmax(outputs/temperature, dim=1)
            predicted = torch.multinomial(soft_out, 1).view(1)
            
            sampled_ids.append(predicted)

            inputs = self.embed(predicted)                       # inputs: (batch_size, embed_size)
            inputs = inputs.unsqueeze(1)                         # inputs: (batch_size, 1, embed_size)
        sampled_ids = torch.stack(sampled_ids, 1)                # sampled_ids: (batch_size, max_seq_length)
        return sampled_ids
    def beam_search_sample(self, features, states=None, beam_width=3):
        """Beam Search를 사용하여 캡션을 생성합니다."""
        inputs = features.unsqueeze(1)  # (batch_size, 1, feature_size)
        batch_size = inputs.size(0)
        
        # 각 배치마다 beam_width만큼의 후보를 관리
        sequences = [[([], 1.0, states)] for _ in range(batch_size)]  # 배치마다 별도의 시퀀스를 관리

        for _ in range(self.max_seg_length):
            all_candidates = [[] for _ in range(batch_size)]
            
            for batch_idx in range(batch_size):
                for seq, score, states in sequences[batch_idx]:
                    hiddens, states = self.lstm(inputs[batch_idx].unsqueeze(0), states)
                    outputs = self.linear(hiddens.squeeze(1))  # (1, vocab_size)
                    soft_out = F.softmax(outputs, dim=1)  # (1, vocab_size)
                    
                    top_k_probs, top_k_words = soft_out.topk(beam_width, dim=1)
                    
                    for i in range(beam_width):
                        candidate_seq = seq + [top_k_words[0][i].item()]
                        candidate_score = score * top_k_probs[0][i].item()
                        candidate_states = (states[0].clone(), states[1].clone())  # 상태 복사
                        
                        all_candidates[batch_idx].append((candidate_seq, candidate_score, candidate_states))
            
            # 상위 beam_width 개의 시퀀스를 선택
            for batch_idx in range(batch_size):
                ordered = sorted(all_candidates[batch_idx], key=lambda x: x[1], reverse=True)
                sequences[batch_idx] = ordered[:beam_width]
            
            # 다음 LSTM 입력 설정
            inputs_list = []
            for batch_idx in range(batch_size):
                inputs_list.append([seq[-1] for seq, _, _ in sequences[batch_idx]])
            inputs = torch.tensor(inputs_list, device=features.device)
            inputs = self.embed(inputs)  # (batch_size, beam_width, embed_size)
            inputs = inputs.view(batch_size * beam_width, 1, -1)  # (batch_size * beam_width, 1, embed_size)
        
        # 배치별로 가장 높은 점수를 받은 시퀀스를 선택
        best_sequences = [sequences[batch_idx][0][0] for batch_idx in range(batch_size)]
        
        # (batch_size, max_seq_length) 형태로 변환하여 반환
        return torch.tensor(best_sequences, device=features.device)
def bleu_n(pred_words_list,label_words_list):

    bleu1 = sentence_bleu([label_words_list], pred_words_list, weights=(1, 0, 0, 0))


# BLEU@2 calculation
    bleu2 = sentence_bleu([label_words_list], pred_words_list, weights=(0, 1, 0, 0))


    bleu3=sentence_bleu([label_words_list], pred_words_list, weights=(0, 0, 1, 0))


    bleu4=sentence_bleu([label_words_list], pred_words_list, weights=(0, 0, 0, 1))
    return bleu1,bleu2,bleu3,bleu4

def rouge_scores(pred_sentence, label_sentence):
    # Initialize the ROUGE scorer
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    
    # Calculate the scores
    scores = scorer.score(label_sentence, pred_sentence)
    
    # Extract the precision, recall, and f1 scores for each ROUGE metric
    rouge1 = scores['rouge1']
    rouge2 = scores['rouge2']
    rougeL = scores['rougeL']
    
    # Return the ROUGE scores
    return rouge1, rouge2, rougeL

In [5]:
with open(params['vocab_path'], 'rb') as f:
        vocab = pickle.load(f)
transform = transforms.Compose([ 
        transforms.RandomCrop(params['image_size']),
        transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(), 
        transforms.Normalize((0.485, 0.456, 0.406), 
                             (0.229, 0.224, 0.225))])

df=pd.read_csv(params['data_path']+params['val_csv'])
test_list=torch.zeros(len(df),3,params['image_size'],params['image_size'])
for i in tqdm(range(len(df))):
    image=tf(Image.open(glob(params['data_path']+'**/**/'+df.loc[i]['path'])[0]).resize((params['image_size'],params['image_size'])))
    test_list[i]=image

test_dataset=CustomDataset(test_list,params['data_path'],params['image_size'],params['val_csv'],'val',vocab,transform=transform)

test_dataloader=DataLoader(test_dataset,batch_size=params['batch_size'],shuffle=True,collate_fn=collate_fn)

 90%|████████▉ | 1646/1836 [03:41<00:25,  7.44it/s]


KeyboardInterrupt: 

In [ ]:

Feature_Extractor=FeatureExtractor()
encoder = AttentionMILModel(params['embed_size'],131072,Feature_Extractor).to(device)
decoder = DecoderRNN(params['embed_size'], params['hidden_size'], len(vocab), params['num_layers']).to(device)
criterion = nn.CrossEntropyLoss()
model_param = list(decoder.parameters()) + list(encoder.parameters())
optimizer = torch.optim.Adam(model_param, lr=params['lr'], betas=(params['beta1'], params['beta2']))
encoder.load_state_dict(torch.load('../../model/captioning/maxvit_BR_encoder_18.pth',map_location=device))
decoder.load_state_dict(torch.load('../../model/captioning/maxvit_BR_decoder_18.pth',map_location=device))


<All keys matched successfully>

In [ ]:
with torch.no_grad():
    val_count=0
    val_loss = 0.0 
    val_bleu_loss=0.0
    val=tqdm(test_dataloader)
    total_bleu=[]
    total_meteor=[]
    encoder.eval()
    decoder.eval()
    total_Rogue=[]
    for images,captions,lengths in val:
        val_count+=1
        images = images.to(device)
        captions = captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
        features = encoder(images)
        outputs = decoder.sample(features)
        outputs = outputs[:,:len(targets)]
        pred_words_list=idx2word(vocab,outputs)
        label_words_list=idx2word(vocab,captions)
        reference = word2sentence(label_words_list)
        candidate = word2sentence(pred_words_list)
        bleu1,bleu2,bleu3,bleu4=bleu_n(pred_words_list,label_words_list)
        rouge1, rouge2, rougeL = rouge_scores(candidate, reference)
        total_bleu.append([bleu1,bleu2,bleu3,bleu4])
        total_Rogue.append([rouge1, rouge2, rougeL])
        val.set_description(f"Step: {val_count+1}")

Step: 1837: 100%|██████████| 1836/1836 [01:53<00:00, 16.16it/s]


In [ ]:

def idx2word(vocab, indices):
    sentence = []
    
    indices=indices.cpu().numpy()
    
    for index in indices:
        word = vocab.idx2word[index.item()]
        sentence.append(word)
    return sentence
with torch.no_grad():
    val_count=0
    val_loss = 0.0 
    val_bleu_loss=0.0
    val=tqdm(test_dataloader)
    total_bleu=[]
    total_meteor=[]
    encoder.eval()
    decoder.eval()
    total_Rogue=[]
    for images,captions,lengths in val:
        val_count+=1
        images = images.to(device)
        captions = captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
        features = encoder(images)
        outputs = decoder(features, captions, lengths)
        outputs = pack_padded_sequence(outputs, lengths, batch_first=True)[0]
        loss = criterion(outputs, targets)
        val_loss+=loss.item()
        
        pred_words_list=idx2word(vocab,torch.argmax(outputs,dim=1))
        label_words_list=idx2word(vocab,captions[0])
        reference = word2sentence(label_words_list)
        candidate = word2sentence(pred_words_list)
        bleu1,bleu2,bleu3,bleu4=bleu_n(pred_words_list,label_words_list)
        rouge1, rouge2, rougeL = rouge_scores(candidate, reference)
        total_bleu.append([bleu1,bleu2,bleu3,bleu4])
        total_Rogue.append([rouge1, rouge2, rougeL])
   
        val.set_description(f"Step: {val_count+1} loss : {val_loss/val_count:.4f} ")

In [ ]:
print(f'Bleu-1:{np.array(total_bleu)[:,0].mean():.4f}+-{np.array(total_bleu)[:,0].std():.4f} \nBleu-2:{np.array(total_bleu)[:,1].mean():.4f}+-{np.array(total_bleu)[:,1].std():.4f} \nBleu-3:{np.array(total_bleu)[:,2].mean():.4f}+-{np.array(total_bleu)[:,2].std():.4f} \nBleu-4:{np.array(total_bleu)[:,3].mean():.4f}+-{np.array(total_bleu)[:,3].std():.4f} \nRogue-1:{np.array(total_Rogue)[:,0].mean():.4f}+-{np.array(total_Rogue)[:,0].std():.4f} \nRogue-2:{np.array(total_Rogue)[:,1].mean():.4f}+-{np.array(total_Rogue)[:,1].std():.4f} \nRogue-L:{np.array(total_Rogue)[:,2].mean():.4f}+-{np.array(total_Rogue)[:,2].std():.4f}')

Bleu-1:0.5264+-0.1163 
Bleu-2:0.2440+-0.1716 
Bleu-3:0.1231+-0.1854 
Bleu-4:0.0804+-0.1852 
Rogue-1:0.4907+-0.1283 
Rogue-2:0.2081+-0.1811 
Rogue-L:0.3720+-0.1576


In [ ]:
anp.array(total_Rogue)[:,0]==np.array(total_Rogue)[:,2]